In [ ]:
import cv2
import numpy as np

In [ ]:
def estimate_line_width(edge_img, line):
    rho, theta = line[0]
    a = np.cos(theta)
    b = np.sin(theta)
    x0 = a * rho
    y0 = b * rho
    x1 = int(x0 + 1000 * (-b))
    y1 = int(y0 + 1000 * (a))
    x2 = int(x0 - 1000 * (-b))
    y2 = int(y0 - 1000 * (a))
    
    # Create a mask of the line
    mask = np.zeros_like(edge_img)
    cv2.line(mask, (x1, y1), (x2, y2), 255, 2)
    
    # Dilate the mask
    dilated_mask = cv2.dilate(mask, None, iterations=2)
    
    overlap = cv2.bitwise_and(dilated_mask, edge_img)
    
    # Estimate width based on overlap
    width = np.sum(overlap) / np.sum(mask)
    
    return width

In [ ]:
# Read the image
image = cv2.imread('path_to_your_image.jpg')
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

In [ ]:
# Use Canny edge detection
edges = cv2.Canny(gray, 50, 150, apertureSize=3)

# Detect lines in the image using Hough Line Transform
lines = cv2.HoughLines(edges, 1, np.pi/180, 200)

In [ ]:
for line in lines:
    rho, theta = line[0]
    a = np.cos(theta)
    b = np.sin(theta)
    x0 = a * rho
    y0 = b * rho
    x1 = int(x0 + 1000 * (-b))
    y1 = int(y0 + 1000 * (a))
    x2 = int(x0 - 1000 * (-b))
    y2 = int(y0 - 1000 * (a))
    
    width = estimate_line_width(edges, line)
    
    # Choose color based on width
    if width < 1.5:
        color = (0, 255, 0)  # Green for thin lines
    else:
        color = (0, 0, 255)  # Red for thick lines
    
    cv2.line(image, (x1, y1), (x2, y2), color, 2)

In [ ]:
# Show the result
cv2.imshow('Detected Lines', image)
cv2.waitKey(0)
cv2.destroyAllWindows()